# Įkeliamos naudotas bibliotekos

In [ ]:
# Standartinės bibliotekos duomenų apdorojimui, saugojimui ir vizualizavimui
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV # Duomenų padalijimo į mokymo ir testavimo rinkinius, hiperparametrų derinimo funkcijos
from sklearn.preprocessing import StandardScaler # Duomenų standartizavimo funkcija
from sklearn.svm import SVC # SVM modelis
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, RocCurveDisplay # Modelio vertinimo metodai

from imblearn.over_sampling import ( # Duomenų augmentavimo metodai
    RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN, KMeansSMOTE
)

# Įkeliami ir apdorojami duomenys
Įkeliami duomenys, užpildomos tuščios reikšmės

In [ ]:
def ikelimas(filepath): # Duomenų įkėlimo funkcija
    # Naudojamas tik 'cs-training.csv' failas iš Kaggle konkurso GiveMeSomeCredit
    # Testavimui galima naudoti mažesnį eilučių skaičių nurodant `nrows` reikšmę
    data = pd.read_csv(filepath) # Duomenų failą galima rasti https://www.kaggle.com/c/GiveMeSomeCredit, naudotas tik 'cs-training.csv' failas
    data = data.iloc[:, 1:]  # Panaikinamas indekso stulpelis
    return data

def uzpildymas(data, columns): # Funkcija skirta trūkstamų reikšmių užpildymui mediana
    for col in columns:
        median_val = data[col].median()
        data[col] = data[col].fillna(median_val)
    return data

# Įkeliami duomenys
data = ikelimas("cs-training.csv")
data = uzpildymas(data, ["NumberOfDependents", "MonthlyIncome"])


# Duomenų vizualizacija (Koreliacijos matrica, stačiakampės diagramos)
Sukuriamos funkcijos koreliacijos matricai ir stačiakampėms diagramoms

In [ ]:
def koreliacijos_matrica(data, method='spearman'): # Funkcija skirta koreliacijos matricos išbrėžimui
    corr_matrix = data.corr(method=method)
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1) # Viršutinės koreliacijos matricos dalies užmaskavimas
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        corr_matrix, 
        mask=mask, 
        annot=True, 
        cmap="flare",
        fmt=".2f", 
        linewidths=0.5
    )
    plt.show()

def staciakampes(data):
    numeric_columns = data.select_dtypes(include=[np.number]).columns[1:]  # Išrenkami tik skaitiniai kintamieji, praleidžiamas pirmas stulpelis su y reikšmėmis
    num_cols = 3
    num_rows = (len(numeric_columns) + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))
    axes = axes.flatten()
    for i, column in enumerate(numeric_columns):
        sns.boxplot(x=data[column], ax=axes[i], color="lightblue")
        axes[i].set_title(f"Stačiakampė diagrama {column}")
        axes[i].set_xlabel(column)
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")
    plt.tight_layout()
    plt.show()

# Išvedimas
koreliacijos_matrica(data)
staciakampes(data)

In [ ]:
def drop_columns(data, columns): # Funkcija skirta smarkiai koreliuojantiems kintamiesiems pašalinti
    return data.drop(columns=columns)

data = drop_columns(data, ["NumberOfTimes90DaysLate", "NumberOfTime60-89DaysPastDueNotWorse"])

# Padalinami duomenys į testavimo ir treniravimo rinkinius
Sukuriama funkcija padalinanti duomenis į testavimo treniravimo rinkinius, standardizuojami duomenys naudojant StandardScaler

In [ ]:
def split_ir_scale(data, target_col, test_size=0.2, random_state=42):
    X = data.drop(columns=[target_col])
    y = data[target_col]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X) # Standardizuojami duomenys
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, random_state=random_state
    )
    return X_train, X_test, y_train, y_test, scaler

X_train, X_test, y_train, y_test, scaler = split_ir_scale(data, "SeriousDlqin2yrs")

# Apribrėžiama funkcija SVM apmokymui su over-sampling metodais, apmokomas SVM ant originalių duomenų
Parašoma funkcija pritaikanti over-sampling metodą, apmokanti SVM ant augmentuotų duomenų, išveda įverčius bei išsaugo AUROC/ROC rezultatus. Funkcija bus naudojoma ir vėlesniuose žingsniuose

In [ ]:
def svm_su_oversampling(
    oversampler, 
    X_train, y_train, 
    X_test, y_test, 
    pavadinimas,
    svm_params = None,
    results_dict=None, 
    displays_dict=None
):
    if svm_params is None:
        svm_params = {}
    if results_dict is None:
        results_dict = {}
    if displays_dict is None:
        displays_dict = {}

    print(f"--- {pavadinimas} ---")
    X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)
    svm_model = SVC(random_state=42, probability=True)
    svm_model.fit(X_resampled, y_resampled)
    y_pred = svm_model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    y_pred_proba = svm_model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    results_dict[pavadinimas] = roc_auc
    displays_dict[pavadinimas] = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=pavadinimas)
    return results_dict, displays_dict

# Apmokomas SVM ant originalių duomenų
auroc_results = {}
roc_displays = {}
svm_model = SVC(random_state=42, probability=True)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print("--- Originalūs duomenys ---")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
y_pred_proba = svm_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)
auroc_results["Originalūs duomenys"] = roc_auc
roc_displays["Originalūs duomenys"] = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name="Originalūs duomenys")

# Over-sampling metodai nepakeistais (nutylėtais) parametrais
oversamplers = [
    (RandomOverSampler(random_state=42), "Random Over-Sampling"),
    (SMOTE(random_state=42), "SMOTE"),
    (BorderlineSMOTE(random_state=42), "Borderline-SMOTE"),
    (SVMSMOTE(random_state=42), "SVM-SMOTE"),
    (ADASYN(random_state=42), "ADASYN"),
    (KMeansSMOTE(random_state=42, k_neighbors=5, cluster_balance_threshold=0.01), "KMeans-SMOTE") #teko pakeisti porą parametrų, kad būtų galima pritaikyti šį metodą be kliūčių
]

for oversampler, name in oversamplers:
    auroc_results, roc_displays = svm_su_oversampling(
        oversampler, X_train, y_train, X_test, y_test, name,  
        results_dict=auroc_results, 
        displays_dict=roc_displays
    )

# GridSearchCV SVM parametrams rasti
Sukuriama funkcija kiekvieno SVM modelio optimaliems parametrams rasti. Pirma pritaikoma originaliems duomenims, tuomet iteruojama per visus over-sampling metodus

In [ ]:
def grid_search_svm(X, y, param_grid=None, subset_ratio=0.2, random_state=42):
    if param_grid is None:
        param_grid = {
            "C": [0.1, 1, 10],
            "gamma": [0.01, 0.1, 1],
            "kernel": ["rbf", "linear"]
        }
    X_gs, _, y_gs, _ = train_test_split(X, y, test_size=1-subset_ratio, random_state=random_state)
    grid_search = GridSearchCV(
        SVC(class_weight="balanced", probability=True, random_state=random_state), 
        param_grid, cv=3
    )
    grid_search.fit(X_gs, y_gs)
    print("Geriausi parametrai:", grid_search.best_params_)
    return grid_search.best_params_

def grid_search_svm_oversamplers(X_train, y_train, oversamplers, param_grid=None, subset_ratio=0.2, random_state=42):
    best_params_dict = {}
    for oversampler, name in oversamplers:
        print(f"\n--- Grid Search su {name} ---")
        X_res, y_res = oversampler.fit_resample(X_train, y_train)
        best_params = grid_search_svm(X_res, y_res, param_grid, subset_ratio, random_state)
        best_params_dict[name] = best_params
    return best_params_dict

# Pateikiami over-sampling metodai su nutylėtais parametrais
oversamplers = [
    (RandomOverSampler(random_state=42), "Random Over-Sampling"),
    (SMOTE(random_state=42), "SMOTE"),
    (BorderlineSMOTE(random_state=42), "Borderline-SMOTE"),
    (SVMSMOTE(random_state=42), "SVM-SMOTE"),
    (ADASYN(random_state=42), "ADASYN"),
    (KMeansSMOTE(random_state=42, k_neighbors=5, cluster_balance_threshold=0.01), "KMeans-SMOTE")
]

print("Grid Search Originaliems duomenims")
best_svm_params = grid_search_svm(X_train, y_train)

print("\nGrid Search kiekvienam over-sampling metodui")
best_params_per_oversampler = grid_search_svm_oversamplers(X_train, y_train, oversamplers)

# GridSearch over-sampling metodų parametrams
Sukuriama funkcija, iteruojanti per kiekvieno over-sampling metodo parametrus ir randanti optimalią kombinaciją

In [ ]:
def grid_search_oversampler(
    oversampler, param_grid, 
    X, y, 
    X_test, y_test, 
    svm_params, 
    pavadinimas, 
    subset_ratio=0.2, 
    random_state=42,
    results_dict=None,
    displays_dict=None
):
    X_subset, _, y_subset, _ = train_test_split(X, y, train_size=subset_ratio, random_state=random_state)
    grid_search = GridSearchCV(oversampler, param_grid, cv=3, scoring='roc_auc', n_jobs=-1) # Parinkti parametrai naudojant GridSearchCV, siekiant optimizuoti vykdymo laiką, bei pakeistas vertinimo kriterijus į 'roc_auc'
    grid_search.fit(X_subset, y_subset)
    print(f"Best Parameters for {pavadinimas}:", grid_search.best_params_)
    best_oversampler = grid_search.best_estimator_
    return svm_su_oversampling(
        best_oversampler, X, y, X_test, y_test, 
        pavadinimas + " (Tuned)", 
        svm_params=svm_params,
        results_dict=results_dict,
        displays_dict=displays_dict
    )

oversampler_param_grids = {
    "Random Over-Sampling": {
        "sampling_strategy": [0.5, 1.0]
    },
    "SMOTE": {
        "sampling_strategy": [0.5, 1.0],
        "k_neighbors": [3, 5, 7]
    },
    "Borderline-SMOTE": {
        "sampling_strategy": [0.5, 1.0],
        "k_neighbors": [3, 5, 7],
        "m_neighbors": [10, 15]
    },
    "SVM-SMOTE": {
        "sampling_strategy": [0.5, 1.0],
        "k_neighbors": [3, 5, 7],
        "m_neighbors": [10, 15]
    },
    "ADASYN": {
        "sampling_strategy": [0.5, 1.0],
        "n_neighbors": [3, 5, 7]
    },
    "KMeans-SMOTE": {
        "sampling_strategy": [0.5, 1.0],
        "k_neighbors": [3, 5, 7],
        "cluster_balance_threshold": [0.01, 0.1]
    }
}

svm_params = { # Po SVM hiperparametrų derinimo, šie parametrai visiems over-sampling metodams buvo nustatyti kaip optimalūs
    "C": 10,
    "gamma": 1,
    "kernel": "rbf"
}

# Sukuriami tušti kintamieji rezultatams ir ROC kreivėms saugoti
auroc_results_tuned = {}
roc_displays_tuned = {}

# Pritaikomas Grid Search kiekvienam over-sampling metodui su hiperparametrų derinimu ir išsaugomi rezultatai
for oversampler, name in oversamplers:
    param_grid = oversampler_param_grids.get(name, None)
    if param_grid is not None:
        auroc_results_tuned, roc_displays_tuned = grid_search_oversampler(
            oversampler.__class__(random_state=42), param_grid, 
            X_train, y_train, X_test, y_test, 
            svm_params, name,
            results_dict=auroc_results_tuned,
            displays_dict=roc_displays_tuned
        )

# Išvedami AUROC rezultatai bei išbrėžiamos ROC kreivės
Sukuriama funkcija, kuri išveda pasirinktus AUROC rezultatus bei išbrėžia ROC kreives visiems over-sampling metodams

In [ ]:
def plot_auroc_rezultatai(auroc_results, roc_displays):
    print("\nAUROC rezultatai:")
    # Išvedami AUROC rezultatai ir brėžiamos ROC kreivės
    for method, auroc in auroc_results.items():
        print(f"{method}: {auroc:.4f}")
    plt.figure(figsize=(10, 8))
    for method, display in roc_displays.items():
        plt.plot(display.fpr, display.tpr, label=f"{method} (AUROC = {display.roc_auc:.4f})")
    plt.plot([0, 1], [0, 1], 'k--', label="Atsitiktinis spėjimas")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_auroc_rezultatai(auroc_results, roc_displays)
plot_auroc_rezultatai(auroc_results_tuned, roc_displays_tuned)